Besonders in eigentlich klassisch geisteswissenschaftlichen Themenfeldern ist es spannend zu sehen, inwieweit moderne computergestützte Verfahren einen Mehrwert und neue Einblicke bieten können. Deshalb schaue ich im folgenden, inwieweit sich regionale Besonderheiten bei der Verteilung von Denkmalen in Hessen zeigen.

Hierfür werde ich das hessische Onlienverzeichnis [DenkXWeb](http://denkxweb.denkmalpflege-hessen.de/) zu Denkmalen in Hessen scrapen, in einer Sqlite Datenbank abspeichern, und dann mittels der folium liberay die unterschiedliche Verteilung von Denkmalen nach Regionen darstellen.

## 1. Vorbereitung

Zunächst werden alle notwendigen librarys geladen. Sqlite3 ermöglicht das erstellen und verwalten von SQL-Datenbanken, pandas ist das Standardwerkzeug, um Daten (vergleichbar zu R) in Dataframes dazustellen und zu bearbeiten. Folium greift auf javascript zurück, um hübsche "Daten-Karten" zu erstellen, also bspw. mittels einer Heatmap auf Landkarten die Häufigkeit einer Eigenschaft zu visualisieren.

In [1]:
import sqlite3
import pandas as pd
import folium
import json
from branca.utilities import _locations_mirror
from folium.features import *

Die bereits gescrapten und abgespeicherten Denkmal-Daten werden mittels pandas geladen. Das zwecks Scraping geschriebene Script findet sich auch im Ordner unter dem Namen Hessen_Scraping.py und nutzt Selenium, um die einzelnen Regionen tabellarisch abzufragen.

In [2]:
conn = sqlite3.connect("Denkmale.db")
df = pd.read_sql_query('''SELECT Denkmale.Id, Ort, Ortsteil, Strasse, Kreis, Flur, Begründung
                          FROM Denkmale''', conn)
df.head()

,Id,Ort,Ortsteil,Strasse,Kreis,Flur,Begründung
0,3,Grünberg,Grünberg,Barfüßergasse 38,Gießen,1,"Der Gewölbekeller, der heute unter einer um od..."
1,423,Bensheim,Bensheim,Nibelungenstraße 100,Bergstraße,13,"Zwischen Bensheim und Schönberg, unmittelbar a..."
2,424,Bensheim,Bensheim,Marktplatz 11,Bergstraße,1,"Eine erste, dem hl. Michael geweihte Kirche in..."
3,425,Bensheim,Bensheim,Hauptstraße 79 Hauptstraße 81Hauptstraße 85Hau...,Bergstraße,1,Mit ihrem Chor westlich an den Hospitalplatz a...
4,426,Bensheim,Bensheim,Friedhofstraße 39 Friedhofstraße 41,Bergstraße,9,Weil man im Zentrum Bensheims Platz für ein ne...


Die Geodaten der einzelnen deutschen Regionen liegen als Json-Datei vor. Die für die Analyse relevanten (= "Hessen") Einträge werden herausgefiltert und gesondert gespeichert.

In [3]:
with open("pyshp-demo.json") as data_file:    
    data = json.load(data_file)
    
hessen_regions = []

for index,entry in enumerate(data["features"]):
    if entry["properties"]["NAME_1"] == "Hessen":
        hessen_regions.append(index)
        
geo_data = [entry["properties"]['NAME_3'] for entry in data["features"]]

## 2. Analyse - Worthäufigkeiten

Im zweiten Schritt erstellen wir mittels sklearn eine Term-Document-Matrix aus den Begründungstexten und bilden daraus einen zweiten Dataframe.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=7000, min_df=20,binary=True)
count_vector = vectorizer.fit_transform(df['Begründung'])

In [5]:
word_count_df = pd.DataFrame(count_vector.toarray())
word_count_df.columns = vectorizer.get_feature_names()
word_count_df.head()

,000,02,03,04,05,06,07,08,09,10,...,überstände,übertragen,überwiegend,überzeugende,üblich,übliche,üblichen,übrige,übrigen,üppig
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


Jetzt fügen wir noch den Dataframe mit den Worthäufigkeiten unserem ursprünglichen Denkmal-Datensatz hinzu.

In [6]:
df_merged = df.join(word_count_df)
df_merged.head()

,Id,Ort,Ortsteil,Strasse,Kreis,Flur,Begründung,000,02,03,...,überstände,übertragen,überwiegend,überzeugende,üblich,übliche,üblichen,übrige,übrigen,üppig
0,3,Grünberg,Grünberg,Barfüßergasse 38,Gießen,1,"Der Gewölbekeller, der heute unter einer um od...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,423,Bensheim,Bensheim,Nibelungenstraße 100,Bergstraße,13,"Zwischen Bensheim und Schönberg, unmittelbar a...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,424,Bensheim,Bensheim,Marktplatz 11,Bergstraße,1,"Eine erste, dem hl. Michael geweihte Kirche in...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,425,Bensheim,Bensheim,Hauptstraße 79 Hauptstraße 81Hauptstraße 85Hau...,Bergstraße,1,Mit ihrem Chor westlich an den Hospitalplatz a...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,426,Bensheim,Bensheim,Friedhofstraße 39 Friedhofstraße 41,Bergstraße,9,Weil man im Zentrum Bensheims Platz für ein ne...,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## 3. Visualisierung - Folium-Maps

Und schon kann die Visualisierung beginnen. Nun können wir bspw. einen Blick darauf werfen, wie groß der Anteil von Fachwerk-Denkmalen in den einzelnen Regionen ist.

In [7]:
df_merged["Fachwerk"] = ["fachwerk" in text.lower() for text in list(df_merged["Begründung"])]

In [8]:
from branca.colormap import linear

column  = "Fachwerk"
df = df_merged


#Die Worthäufigkeiten werden nach orten aggregiert
group_df = df.groupby(["Ort"])[column].mean()
group_df = group_df.reset_index()
group_df_dict = group_df.set_index('Ort')[column]
ort_list = list(group_df_dict.index)

#Laden der Geodaten zwecks Darstellung der Regionen/Kreise
for ort in ort_list:
    if ort not in geo_data:
        print("Ort: {} nicht in Geodaten".format(ort))
with open("pyshp-demo.json") as data_file:    
    data = json.load(data_file)

#Es werden nur relevante(hessische) Kreisdaten beibehalten
hessen_regions = []
for index,entry in enumerate(data["features"]):
    if entry["properties"]["NAME_1"] == "Hessen" and entry["properties"]["NAME_3"] in ort_list:
        hessen_regions.append(index)

#Umwandlung in folium-konformes Format
data["features"] = [data["features"][index] for index in hessen_regions]
colormap = linear.YlGn.scale(group_df[column].min(), group_df[column].max())
geojson = [{'type': data['type'], 'features': [f]} for f in data['features']]

#Bilden der Folium Map
m = folium.Map(location=[50.652051,9.162438], zoom_start=8,tiles='Mapbox Bright')
folium.GeoJson(data, name=column,style_function=lambda feature: { 
                                                    'fillColor': colormap(group_df_dict[feature["properties"]['NAME_3']]),
                                                    'weight': 1,
                                                    'dashArray': '5, 5',
                                                    'fillOpacity': 0.9,  
                                                    'color': 'black',                                                            
                                                                }).add_to(m)
colormap.caption = column
colormap.add_to(m)
#for gj in map(lambda gj: folium.GeoJson(gj), geojson):
    #props = gj.data['features'][0]['properties']["NAME_3"]
    #gj.add_child(folium.Popup(str(props)))
    #gj.add_to(m)
m

Leider werden die Karten in JupyterNotebooks nicht mehr dargestellt, sobald Popups mit Namen der einzelnen Regionen hinzufüget werden. Deshalb ist dieser Teil auskommentiert. Bei Interesse liegen die Karten als Html-Dokument inklusive Popups im Ordner bei, sodass auch nachvollziogen werden kann, um welche Regionen es sich genau handelt.

Es zeigt sich zum einen, das mehr als die Hälfte Hessens noch ein weißer Fleck ist, was das DenkmalXWeb angeht. Bei den Regionen, für die die Denkmalbeschreibungen vorliegen lässt sich ein nachvollziehbarer Trend erkennen: Frankfurt direkt hat den geringsten Anteil an Fachwerkdenkmalen, während gerade die abgebildeten (ländlicheren) mittelhessischen Gebiete hauptsächlich durch Fachwerk in der Denkmaldatenbank vertreten sind. Wie sieht es mit einem speziellen Baustoff wie Schiefer aus?

In [9]:
df_merged["Schiefer"] = ["schiefer" in text.lower() for text in list(df_merged["Begründung"])]

In [10]:
column  = "Schiefer"
df = df_merged

group_df = df.groupby(["Ort"])[column].mean()
group_df = group_df.reset_index()
group_df_dict = group_df.set_index('Ort')[column]
ort_list = list(group_df_dict.index)

for ort in ort_list:
    if ort not in geo_data:
        print("Ort: {} nicht in Geodaten".format(ort))
with open("pyshp-demo.json") as data_file:    
    data = json.load(data_file)
    
hessen_regions = []
for index,entry in enumerate(data["features"]):
    if entry["properties"]["NAME_1"] == "Hessen" and entry["properties"]["NAME_3"] in ort_list:
        hessen_regions.append(index)

data["features"] = [data["features"][index] for index in hessen_regions]
colormap = linear.YlGn.scale(group_df[column].min(), group_df[column].max())
geojson = [{'type': data['type'], 'features': [f]} for f in data['features']]

m = folium.Map(location=[50.652051,9.162438], zoom_start=8,tiles='Mapbox Bright')
folium.GeoJson(data, name=column,style_function=lambda feature: { 
                                                    'fillColor': colormap(group_df_dict[feature["properties"]['NAME_3']]),
                                                    'weight': 1,
                                                    'dashArray': '5, 5',
                                                    'fillOpacity': 0.9,  
                                                    'color': 'black',                                                            
                                                                }).add_to(m)
colormap.caption = column
colormap.add_to(m)
#for gj in map(lambda gj: folium.GeoJson(gj), geojson):
    #props = gj.data['features'][0]['properties']["NAME_3"]
    #gj.add_child(folium.Popup(str(props)))
    #gj.add_to(m)
m

Die Verteilung von Denkmalen, bei denen "Schiefer" im Begründungstext auftaucht zeigt zwei klare Zentren, in denen dieser Typus gehäuft vorkommt. Auch die umgebenden Gebiete scheinen häufiger mit Schiefer zu bauen, möglicherweise lässt das auf regionale Schiefervorkommen schließen.